In [1]:
# Method for pre-processing x-ray images and generating knee joint dataset

%matplotlib inline

import random as rand
#import pandas as pd
import numpy as np
#from PIL import Image
#import dicom
import matplotlib.pyplot as plt
import scipy.ndimage as ndimage

# Functions to Preprocess Images

In [2]:
# Methods for preprocessing the images 
# Changes resolution, rescales pixel values, changes MONOCHROME1, crops approximate knee joint areas

'''
    Def:
        Interpolate image to get a fixed resolution - we use a scaling factor of 0.2
    Params:
        image_dicom = dicom of xray images
        scaling_factor = factor to scale resolution
    Return:
        image_array = array for image with scaled resolution
'''
def interpolate_resolution(image_dicom, scaling_factor):
    
    image_array = image_dicom.pixel_array
    
    x = image_dicom[0x28, 0x30].value[0]
    y = image_dicom[0x28, 0x30].value[1]
    
    image_array = ndimage.zoom(image_array, [x/scaling_factor, y/scaling_factor])
    return image_array


'''
    Def:
        Function for re-scaling images that have (0028, 1050) and (0028, 1051) dicom headers
        Re-scale the pixel values using 
        Method from ftp://dicom.nema.org/MEDICAL/dicom/2014c/output/chtml/part03/sect_C.11.2.html#sect_C.11.2.1.2)
    Params:
        image_dicom = dicom of xray images
        image_array = array for an image
    Return:
        image_array = array for image that has pixel values rescaled using a different method
'''
def scale_pixel_values_using_cw(image_dicom, image_array):
    
    c = image_dicom[0x28, 0x1050].value
    w = image_dicom[0x28, 0x1051].value
    
    image_array = np.where(image_array <= (c - 0.5 - (w-1)/2), 0,  
                           np.where(image_array > c - 0.5 + (w-1)/2, 255.0, 
                                    np.float32(((image_array - (c-0.5))/(w-1) + 0.5) * 255.0 )))
    return image_array


'''
    Def:
        Function for re-scaling images that have type LOG_E REL for header (0028, 1054)
    Params:
        image_dicom = dicom of xray images
        image_array = array for an image
    Return:
        image_array = array for image that has pixel values rescaled using a logerel method
'''
def scale_pixel_values_using_logerel(image_dicom, image_array):
    
    intercept = image_dicom[0x28, 0x1052].value
    slope = image_dicom[0x28, 0x1053].value
    
    # For edge cases w/o intercept or slope values, then we will force the use of intercept = 0.0, slope = 1.0
    if isinstance(intercept, str):
        intercept = 0.0
        slope = 1.0
        print(image_dicom[0x10, 0x20].value + ' is an edge  case for pixel scaling')
    
    # Linear transformation of data w/ slope and intercept values
    image_array = slope * image_array + intercept
    
    minimum = np.min(image_array)
    maximum = np.max(image_array)
    
    # Re-scale to new range of 0-255
    image_array = np.float32(255.0 * (image_array - minimum) / (maximum - minimum))
    return image_array


'''
    Def:
        Function for inverting images with MONOCHROME1 after the images have been converted to same pixel scale
    Params:
        image_array = array for an image
    Return:
        image_array = image with monochrome1 inverted
'''
def invert_Monochrome1(image_array):
    image_array = -image_array + 255.0
    return image_array


'''
    Def:
        Method to automatically extract knee joint
        For parameter side: 1 = right, 0 = left
    Params:
        image_array = array for an image
        side = side of knee
    Return:
        image_array = cropped knee joint array
'''
def extract_knee(image_array, side):
    
    print('Dimensions of image: ', image_array.shape)
    
    # Compute the sum of each row and column
    col_sums = np.sum(image_array, axis = 0)
    row_sums = np.sum(image_array, axis = 1)
    
    # Row index for cropping is centered at the minimum of the row_sums array
    row_start = np.argmin(row_sums)-512
    row_end = np.argmin(row_sums)+512
    print('Row Indices Original: ', row_start, row_end)
    
    # However, if either the start or end of the row values is beyond the original image array shape
    # We center the cropped image at the center row of the original image array
    if row_start < 0 or row_end > (image_array.shape[0]-1):
            row_start = round(image_array.shape[0]/2)-512
            row_end = round(image_array.shape[0]/2)+512
            
    print('Row Indices Final: ', row_start, row_end)
    
    # For right knee, crop columns to be centered at the maximum sum of the LHS of original image array 
    # Shift over by 500 columns in edge cases with white outer bars   
    if side == 1:
        col_center = 500+np.argmax(col_sums[500:round(col_sums.shape[0]/2)])
        print('Column Indices for Right Original: ', col_center-512, col_center+512)
        
        # If column is below original image array size, then start cropping on left hand border and go out 1024 columns
        if (col_center - 512) < 0:
            print('Column Indices for Right Final: ', 0, 1024)
            image_array = image_array[row_start:row_end, :1024]           
            
        else:
            image_array = image_array[row_start:row_end, (col_center-512):(col_center+512)]
            print('Column Indices for Right Final: ', col_center-512, col_center+512)
            
    # For left knee, crop columns to be centered at the maximum sum of the RHS of original image array 
    # Shift over by 500 columns in edge cases with white outer bars
    if side == 0:
        col_center = round(col_sums.shape[0]/2)+np.argmax(col_sums[round(col_sums.shape[0]/2):col_sums.shape[0]-500])
        print('Column Indices for Left Original: ', col_center-512, col_center+512)
       
        # If column is above original image array size, then start cropping on right hand border and go in 1024 columns
        if (col_center + 512) > (image_array.shape[1]-1):
            print('Column Indices for Left Final: ', image_array.shape[1]-1024, image_array.shape[1]-1)
            image_array = image_array[row_start:row_end, image_array.shape[1]-1024:]
            
        else:
            image_array = image_array[row_start:row_end, (col_center-512):(col_center+512)]
            print('Column Indices for Left Final: ', col_center-512, col_center+512)
    return image_array


'''
    Def:
        Take right or left side of image - does not crop 1024x1024 sized images but just slices the image into right / left halves
        1 = right, 0 = left
    Params:
        image_array = array for an image
        side = right or left
    Returns:
        image_array = cropped right / left side of knee xray
'''
def take_side(image_array, side):
    
    if side == 1:
        image_array = image_array[:, :round(image_array.shape[1]/2)]
        
    if side == 0:
        image_array = image_array[:, round(image_array.shape[1]/2):]
        
    return image_array


'''
    Def:
        Method for pre-processing the images, which combines the above methods
        1. Re-scale image depending on what DICOM header information is available
        2. If image is MONOCHROME1 - invert to MONOCHROME2 scale
        3. Interpolate image with a new pixel scaling factor of: (from DICOM) original-pixel-width / 0.2
    Params:
        image_dicom = dicom of image (information with image array)
    Return:
        image_array = preprocessed image
'''
def augment_xrays(image_dicom):
    
    image_array = interpolate_resolution(image_dicom, 0.2)
    
    if (0x28, 0x1050) in image_dicom and (0x28, 0x1051) in image_dicom:
        image_array = scale_pixel_values_using_cw(image_dicom, image_array)
        print('Scaled with 28, 1050 and 28, 1051 method')
    
    else: 
        image_array = scale_pixel_values_using_logerel(image_dicom, image_array)
        print('Scaled with Linear Transform')
        
    if image_dicom[0x28, 0x04].value == "MONOCHROME1":
        image_array = invert_Monochrome1(image_array)
        print(image_dicom[0x10, 0x20].value, image_dicom[0x28, 0x04].value)
        
    return image_array 


'''
    Def:
        Writes the image to disk (from Cem)
    Params:
        img = the rgb image to save
        path = the target path
'''
def save_image(img, path):
    
    Image.fromarray(img.round().astype(np.uint8)).save(path, 'jpeg', dpi=[300, 300], quality=90)


'''
    Def:
        Converts the given array into a RGB image. If the number of channels is not
        3 the array is tiled such that it has 3 channels
    Params:
        img = the array to convert [nx, ny, channels]
    Return:
        img = the rgb image [nx, ny, 3]
'''
def to_rgb(img):
   
    img = np.atleast_3d(img)
    channels = img.shape[2]
    if channels < 3:
        img = np.tile(img, 3)
    return img


'''
    Def:
        Given a data frame of image id and side, preprocesses images (rescale, invert, interpolate, crop)
        Other params denote which images we want (which visit (list of strings) and which cohort (a single string)) 
    Params:
        data = list of IDs
        months = list of months id
        cohort = name of cohort
    Return:
        final_array_1024 = list with 2 arrays:
                                1. patient information with id, label, side
                                2. image pixels
'''
def fully_preprocess_xrays(data, months, cohort):
    
    # Create 2 Numpy arrays - one to encode patient information and the others to store the images (1024 and 224 scales)
    # For array of patient information: 
    # 1st column = id, 2nd column = indicator for TKR (1) or no TKR (0), 3rd column = side - right (1) or left (0)
    array_of_information = np.array(data.iloc[:,[1,6,7]])
    array_of_images_1024 = np.empty([len(data), 1024, 1024], dtype=np.float32)
    
    # Iterate over the list of month codes
    for j in months:
        for i in range(len(data)):
            # path where DICOM images are stored
            path = 'R:\\denizlab\\denizlabspace\\Users\\Kevin_Leung\\2017-07-26 Kevin\\' + cohort + ' raw\\' + str(data.iloc[i,1]) + '\\xray\\' + j
            print(i, path)
            image_dicom = dicom.read_file(path)
            
            # Entire image preprocessing procedure
            image_array = augment_xrays(image_dicom)
            image_array_1024 = extract_knee(image_array, data.iloc[i,7])
            print('Min and Max of Full Image:    ', np.min(image_array), np.max(image_array))
            print('Min and Max of Cropped Image: ', np.min(image_array_1024), np.max(image_array_1024))
            print()
            
            # Store the arrays in corresponding parent arrays
            array_of_images_1024[i,:,:] = image_array_1024
            
            # Paths to save jpegs for images
            preprocessed_image_file = str(data.iloc[i,1]) + '_label_' + str(data.iloc[i,6]) + '_side_' + str(data.iloc[i,7]) + '_month_' + j +'.jpeg'  
            preprocessed_path_1024 = 'R:\\denizlab\\denizlabspace\\Users\\Kevin_Leung\\2017-07-26 Kevin\\' + cohort + ' preprocessed\\jpegs_of_preprocessed_knees_1024x1024\\' + preprocessed_image_file  
            
            # First convert to rgb for jpeg format then save in the above path
            image_to_save_1024 = to_rgb(image_array_1024)
            save_image(image_to_save_1024, preprocessed_path_1024)
                    
    # Final array to return combines the numpy array for patient information and the preprocessed image arrays                
    final_array_1024 = [array_of_information, array_of_images_1024]
    return final_array_1024  

In [3]:
# Methods for randomly separating 1024x1024 images into train, validation, and test sets

'''
    Def:
        Randomize original datasets
    Params:
        patient_info = array of patient information with id, label, side
        images = array of images
        dim_1 = size of first dimension
        dim_2 = size of second dimension
    Return:
        randomized_patient_info = patient information array that is randomly scrambled
        randomized_images = patient images that are randomly scrambled
'''
def randomized_data(patient_info, images, dim_1, dim_2):
    
    randomized_indices = rand.sample(range(728), 728)
    randomized_patient_info = np.zeros([728,3])
    randomized_images = np.zeros([728, dim_1, dim_2])
    
    for i in range(728):
        randomized_patient_info[i] = patient_info[randomized_indices[i]]
        randomized_images[i] = images[randomized_indices[i]]
        
    return randomized_patient_info, randomized_images


'''
    Def:
        Separate data set into training, validation, and test sets
        First randomly shuffle vector of 0-727 which will be used as the indices for our respective datasets
    Params:
        patient_info = array of patient information with id, label, side
        images = array of images
        dim_1 = size of first dimension
        dim_2 = size of second dimension
    Return:
        training_data = training set after scrambled
        validation_data = validation set after scrambled
        test_data = test set after scrambled
'''
def separate_sets_v2(patient_info, images, dim_1, dim_2):
    
    randomized_patient_info, randomized_images = randomized_data(patient_info, images, dim_1, dim_2)
    
    training_patient_info = np.zeros([510, 3])
    validation_patient_info = np.zeros([74, 3])
    test_patient_info = np.zeros([144, 3])
    
    training_images = np.zeros([510, dim_1, dim_2])
    validation_images = np.zeros([74, dim_1, dim_2])
    test_images = np.zeros([144, dim_1, dim_2])
    
    j = 0
    k = 0
    l = 0

    for i in range(728):
        if randomized_patient_info[i,1] == 1:
            if np.sum(training_patient_info[:,1]) < 255:
                training_patient_info[j] = randomized_patient_info[i]
                training_images[j] = randomized_images[i]
                j += 1
            
            elif np.sum(validation_patient_info[:,1]) < 37:
                validation_patient_info[k] = randomized_patient_info[i]
                validation_images[k] = randomized_images[i] 
                k += 1
            
            elif np.sum(test_patient_info[:,1]) < 72:
                test_patient_info[l] = randomized_patient_info[i]
                test_images[l] = randomized_images[i]
                l += 1
                
        elif randomized_patient_info[i,1] == 0:
            if j - np.sum(training_patient_info[:,1]) < 255:
                training_patient_info[j] = randomized_patient_info[i]
                training_images[j] = randomized_images[i]
                j += 1
            
            elif k - np.sum(validation_patient_info[:,1]) < 37:
                validation_patient_info[k] = randomized_patient_info[i]
                validation_images[k] = randomized_images[i] 
                k += 1
            
            elif l - np.sum(test_patient_info[:,1]) < 72:
                test_patient_info[l] = randomized_patient_info[i]
                test_images[l] = randomized_images[i]
                l += 1
                
    training_data = [training_patient_info, training_images]
    validation_data = [validation_patient_info, validation_images]
    test_data = [test_patient_info, test_images]
        
    return training_data, validation_data, test_data

In [4]:
'''
    Def:
        Re-scale images to 224x224 from 1024x1024 in order to transfer learn from ResNet
        Due to the interpolation, ranges can now be outside of 0-255, 
        so any negative values are pulled to 0 and anything over 255 is forced to 255 
    Params:
        all_images = array of all images
        cohort = name of cohort
        dim1 = size of dimension 1
        dim2 = size of dimension 2
        dim3 = size of dimension 3
    Return:
        final_array_224 = images resized to 224x224
'''
def rescale_to_224_original(all_images, cohort, dim1, dim2, dim3): 
    
    array_of_images_224 = np.empty([all_images[1].shape[0],  dim1, dim2, dim3], dtype=np.float32)
    
    for i in range(all_images[1].shape[0]):
        preprocessed_image_file = str(all_images[0][i,0]) + '_label_' + str(all_images[0][i,1]) + '_side_' + str(all_images[0][i,2]) + '_month_00' +'.jpeg'  
        preprocessed_path_224 = 'R:\\denizlab\\denizlabspace\\Datasets\\OAI\\2017-07-26 Kevin\\' + cohort + ' preprocessed\\jpegs_of_preprocessed_knees_224x224\\' + preprocessed_image_file
        
        image_array_224 = ndimage.zoom(all_images[1][i,:,:], [dim1/1024.0, dim2/1024.0])
        print(i, all_images[0][i,0], "Original Range:", np.min(image_array_224), np.max(image_array_224))
        
        image_array_224 = np.abs(np.where(image_array_224 < 0, 0.0,  
                                          np.where(image_array_224 > 255.0, 255.0, image_array_224)))
        print(i, all_images[0][i,0], "Re-Scaled Range:", np.min(image_array_224), np.max(image_array_224))
        print()
        
        # Images are converted to have 3-color channels (done by just stacking the single array 3 times)
        image_array_224 = to_rgb(image_array_224)
        array_of_images_224[i] = image_array_224    
        save_image(image_array_224, preprocessed_path_224)
        
    final_array_224 = [all_images[0], array_of_images_224]
    return final_array_224

# Pre-processing X-Rays

In [5]:
import pandas as pd
import os
import sys

In [6]:
sys.path.append('../')
from src.utils import *



ModuleNotFoundError: No module named 'pydicom'

In [ ]:
content_file_path='/gpfs/data/denizlab/Datasets/OAI_original/'
month = '00m'
method = 'mean',
save_dir = '/gpfs/data/denizlab/Users/bz1030/test/test1/'

In [ ]:
content_file_path = os.path.join(content_file_path,month)
file_name = 'contents.csv'
count = 0